### HADOOP COMMANDS
### 1. START HADOOP
##### allstart.sh
### 2. COPY FILE FROM LOCAL TO HADOOP
#### hadoop fs -copyFromLocal Corona_NLP_train.csv
### 3. OPEN JUPYTER NOTEBOOK 
##### pysparknb

### Importing Libraries

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
import numpy as np

from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml import Pipeline

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import StopWordsRemover

from pyspark.ml.feature import CountVectorizer

In [ ]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import time

### Import Data

In [ ]:
df = spark.read.csv("dbfs:/FileStore/shared_uploads/joel.ignatius@centurylink.com/Corona_NLP_train.csv", sep=",", header=True, inferSchema=True)
# df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/joel.ignatius@centurylink.com/Corona_NLP_train.csv")

### Data Exploration

In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
 UserName| ScreenName| Location| TweetAt| OriginalTweet|Sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
 3799| 48751| London| 16-03-2020|@MeNyrbie @Phil_G...| Neutral|
 3800| 48752| UK| 16-03-2020|advice Talk to yo...| Positive|
 3801| 48753| Vagabonds| 16-03-2020|Coronavirus Austr...| Positive|
 3802| 48754| null| 16-03-2020|My food stock is ...| null|
 PLEASE| don't panic| THERE WILL BE EN...| null| null| null|
 Stay calm| stay safe.| null| null| null| null|
#COVID19france #C...| Positive| null| null| null| null|
 3803| 48755| null| 16-03-2020|Me, ready to go a...| null|
Not because I'm p...| but because my f...| but please| don't panic. It ...| null| null|
#CoronavirusFranc...| Extremely Negative| null| null| null| null|
 3804| 48756|ÜT: 36.319708,-82...| 16-03-2020|As news of the re...| Positive|
 3805| 48757|35.926541,-78.753267| 16-03-2020|"Cashier at groce...| Positive|
 3806| 48758| Austria| 16-03-2020|Was at the superm...| null|
#toiletpapercrisi...| Neutral| null| null| null| null|
 3807| 48759| Atlanta, GA USA| 16-03-2020|Due to COVID-19 o...| Positive|
 3808| 48760| BHAVNAGAR,GUJRAT| 16-03-2020|For corona preven...| Negative|
 3809| 48761| Makati, Manila| 16-03-2020|All month there h...| Neutral|
 3810| 48762|Pitt Meadows, BC,...| 16-03-2020|Due to the Covid-...| null|
The wait time may...| particularly bee...| null| null| null| null|
We thank you for ...| Extremely Positive| null| null| null| null|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
only showing top 20 rows

In [ ]:
df.columns

Out[19]: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']

In [ ]:
len(df.columns)

Out[20]: 6

In [ ]:
df.printSchema()

root
-- UserName: string (nullable = true)
-- ScreenName: string (nullable = true)
-- Location: string (nullable = true)
-- TweetAt: string (nullable = true)
-- OriginalTweet: string (nullable = true)
-- Sentiment: string (nullable = true)

In [ ]:
df.head()

Out[22]: Row(UserName='3799', ScreenName='48751', Location='London', TweetAt='16-03-2020', OriginalTweet='@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8', Sentiment='Neutral')

In [ ]:
df.tail(3)

Out[23]: [Row(UserName='44954', ScreenName='89906', Location=None, TweetAt='14-04-2020', OriginalTweet='Is it wrong that the smell of hand sanitizer is starting to turn me on?\n', Sentiment=None),
 Row(UserName='#coronavirus #COVID19 #coronavirus"', ScreenName='Neutral', Location=None, TweetAt=None, OriginalTweet=None, Sentiment=None),
 Row(UserName='44955', ScreenName='89907', Location='i love you so much || he/him', TweetAt='14-04-2020', OriginalTweet="@TartiiCat Well new/used Rift S are going for $700.00 on Amazon rn although the normal market price is usually $400.00 . Prices are really crazy right now for vr headsets since HL Alex was announced and it's only been worse with COVID-19. Up to you whethe", Sentiment='Negative')]

In [ ]:
df.count()

Out[24]: 68046

In [ ]:
df.describe().show()

+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+
summary| UserName| ScreenName| Location| TweetAt| OriginalTweet| Sentiment|
+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+
 count| 68042| 55629| 34247| 41735| 41383| 28617|
 mean| 8.975066046523207E7| 150387.47623557984| 1.76800437504E10| 10.0| 682.0| 1016.0|
 stddev| 9.10012357766514E9|1.645384113616883E7|8.839999088543759E10| NaN| 1176.0629234866644| 1418.4562030602144|
 min| ...| | | | Coronavirus...| "" Well covid-19...|
 max|Ø With April 1 d...|  but for us|ï? ???????'? ????...| she says."|Ø As buyers stoc...| when I also rece...|
+-------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+

In [ ]:
df.select('TweetAt','OriginalTweet','Sentiment').show()

+--------------------+--------------------+---------+
 TweetAt| OriginalTweet|Sentiment|
+--------------------+--------------------+---------+
 16-03-2020|@MeNyrbie @Phil_G...| Neutral|
 16-03-2020|advice Talk to yo...| Positive|
 16-03-2020|Coronavirus Austr...| Positive|
 16-03-2020|My food stock is ...| null|
 null| null| null|
 null| null| null|
 null| null| null|
 16-03-2020|Me, ready to go a...| null|
 don't panic. It ...| null| null|
 null| null| null|
 16-03-2020|As news of the re...| Positive|
 16-03-2020|"Cashier at groce...| Positive|
 16-03-2020|Was at the superm...| null|
 null| null| null|
 16-03-2020|Due to COVID-19 o...| Positive|
 16-03-2020|For corona preven...| Negative|
 16-03-2020|All month there h...| Neutral|
 16-03-2020|Due to the Covid-...| null|
 null| null| null|
 null| null| null|
+--------------------+--------------------+---------+
only showing top 20 rows

In [ ]:
df = df.select('TweetAt','OriginalTweet','Sentiment')

In [ ]:
df.toPandas()['OriginalTweet'].isnull().sum()

Out[28]: 26663

In [ ]:
df = df.dropna(subset=('OriginalTweet'))

In [ ]:
df.show()

+----------+--------------------+------------------+
 TweetAt| OriginalTweet| Sentiment|
+----------+--------------------+------------------+
16-03-2020|@MeNyrbie @Phil_G...| Neutral|
16-03-2020|advice Talk to yo...| Positive|
16-03-2020|Coronavirus Austr...| Positive|
16-03-2020|My food stock is ...| null|
16-03-2020|Me, ready to go a...| null|
16-03-2020|As news of the re...| Positive|
16-03-2020|"Cashier at groce...| Positive|
16-03-2020|Was at the superm...| null|
16-03-2020|Due to COVID-19 o...| Positive|
16-03-2020|For corona preven...| Negative|
16-03-2020|All month there h...| Neutral|
16-03-2020|Due to the Covid-...| null|
16-03-2020|#horningsea is a ...|Extremely Positive|
16-03-2020|Me: I don't need ...| null|
16-03-2020|ADARA Releases CO...| Positive|
16-03-2020|Lines at the groc...| null|
16-03-2020|????? ????? ?????...| null|
16-03-2020|"@eyeonthearctic ...| null|
16-03-2020|Amazon Glitch Sty...| null|
16-03-2020|For those who are...| Positive|
+----------+--------------------+------------------+
only showing top 20 rows

In [ ]:
df.toPandas()['Sentiment'].isnull().sum()

Out[31]: 12766

In [ ]:
df = df.dropna(subset=('Sentiment'))

In [ ]:
df.show()

+----------+--------------------+------------------+
 TweetAt| OriginalTweet| Sentiment|
+----------+--------------------+------------------+
16-03-2020|@MeNyrbie @Phil_G...| Neutral|
16-03-2020|advice Talk to yo...| Positive|
16-03-2020|Coronavirus Austr...| Positive|
16-03-2020|As news of the re...| Positive|
16-03-2020|"Cashier at groce...| Positive|
16-03-2020|Due to COVID-19 o...| Positive|
16-03-2020|For corona preven...| Negative|
16-03-2020|All month there h...| Neutral|
16-03-2020|#horningsea is a ...|Extremely Positive|
16-03-2020|ADARA Releases CO...| Positive|
16-03-2020|For those who are...| Positive|
16-03-2020|with 100 nations...|Extremely Negative|
16-03-2020|@10DowningStreet ...| Negative|
16-03-2020|UK #consumer poll...|Extremely Positive|
16-03-2020|In preparation fo...| Negative|
16-03-2020|This morning I te...|Extremely Negative|
16-03-2020|There Is of in th...| Negative|
16-03-2020|Went to the super...| Neutral|
16-03-2020|Worried about the...| Positive|
16-03-2020|my wife works ret...| Negative|
+----------+--------------------+------------------+
only showing top 20 rows

In [ ]:
import re
from pyspark.sql.functions import regexp_replace

In [ ]:
df.withColumn("OriginalTweet", regexp_replace(col("OriginalTweet"), "/[^0-9A-Za-z t]+/" , ""))

Out[35]: DataFrame[TweetAt: string, OriginalTweet: string, Sentiment: string]

### Feature Engineering

In [ ]:
import pyspark.ml.feature
from pyspark.ml.feature import IDF

In [ ]:
####Initialising the pipeline stages
tokenizer = Tokenizer(inputCol='OriginalTweet' , outputCol='words')
stopwords_remover = StopWordsRemover(inputCol='words', outputCol='filtered_words')
vectorizer = CountVectorizer(inputCol='filtered_words' , outputCol='vector_words')
idf = IDF(inputCol='vector_words' , outputCol='vectorized_features')

In [ ]:
####Adding Labels
labelEncoder = StringIndexer(inputCol='Sentiment' , outputCol='label').fit(df)

In [ ]:
labelEncoder.transform(df).show(10)

+----------+--------------------+------------------+-----+
 TweetAt| OriginalTweet| Sentiment|label|
+----------+--------------------+------------------+-----+
16-03-2020|@MeNyrbie @Phil_G...| Neutral| 2.0|
16-03-2020|advice Talk to yo...| Positive| 0.0|
16-03-2020|Coronavirus Austr...| Positive| 0.0|
16-03-2020|As news of the re...| Positive| 0.0|
16-03-2020|"Cashier at groce...| Positive| 0.0|
16-03-2020|Due to COVID-19 o...| Positive| 0.0|
16-03-2020|For corona preven...| Negative| 1.0|
16-03-2020|All month there h...| Neutral| 2.0|
16-03-2020|#horningsea is a ...|Extremely Positive| 3.0|
16-03-2020|ADARA Releases CO...| Positive| 0.0|
+----------+--------------------+------------------+-----+
only showing top 10 rows

In [ ]:
df = labelEncoder.transform(df)

In [ ]:
df.show()

+----------+--------------------+------------------+-----+
 TweetAt| OriginalTweet| Sentiment|label|
+----------+--------------------+------------------+-----+
16-03-2020|@MeNyrbie @Phil_G...| Neutral| 2.0|
16-03-2020|advice Talk to yo...| Positive| 0.0|
16-03-2020|Coronavirus Austr...| Positive| 0.0|
16-03-2020|As news of the re...| Positive| 0.0|
16-03-2020|"Cashier at groce...| Positive| 0.0|
16-03-2020|Due to COVID-19 o...| Positive| 0.0|
16-03-2020|For corona preven...| Negative| 1.0|
16-03-2020|All month there h...| Neutral| 2.0|
16-03-2020|#horningsea is a ...|Extremely Positive| 3.0|
16-03-2020|ADARA Releases CO...| Positive| 0.0|
16-03-2020|For those who are...| Positive| 0.0|
16-03-2020|with 100 nations...|Extremely Negative| 4.0|
16-03-2020|@10DowningStreet ...| Negative| 1.0|
16-03-2020|UK #consumer poll...|Extremely Positive| 3.0|
16-03-2020|In preparation fo...| Negative| 1.0|
16-03-2020|This morning I te...|Extremely Negative| 4.0|
16-03-2020|There Is of in th...| Negative| 1.0|
16-03-2020|Went to the super...| Neutral| 2.0|
16-03-2020|Worried about the...| Positive| 0.0|
16-03-2020|my wife works ret...| Negative| 1.0|
+----------+--------------------+------------------+-----+
only showing top 20 rows

In [ ]:
### Positive = 0.0
### Negative = 1.0
### Neutral  = 2.0
### Extermely Positive = 3.0
### Extremely Negative = 4.0

### Model

In [ ]:
train,test = df.randomSplit([0.6,0.4])

In [ ]:
lr = LogisticRegression(featuresCol='vectorized_features' , labelCol='label')

### Pipeline

In [ ]:
pipeline = Pipeline(stages = [tokenizer, stopwords_remover, vectorizer, idf, lr])

### Building Model

In [ ]:
lr_model = pipeline.fit(train)

In [ ]:
### Testing Model
predictions = lr_model.transform(test)

In [ ]:
predictions.show()

+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 TweetAt| OriginalTweet| Sentiment|label| words| filtered_words| vector_words| vectorized_features| rawPrediction| probability|prediction|
+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 #washingtonpost | #latimes https:/...| Neutral| 2.0|[, #latimes, http...|[, #latimes, http...|(55598,[1,13978],...|(55598,[1,13978],...|[-0.9056172425248...|[2.45415495018404...| 2.0|
 Distribution Staff| Lorry Drivers| Utilities etc"|102.0| [, lorry, drivers]| [, lorry, drivers]|(55598,[1,262,434...|(55598,[1,262,434...|[14.7389659008539...|[0.03840908619689...| 1.0|
 Media| Actors| Teachers| 99.0| [, actors]| [, actors]|(55598,[1,5062],[...|(55598,[1,5062],[...|[17.2510706247510...|[0.84862043037986...| 0.0|
 Starbucks| Black Lives Matter| Woodstock sanctuary|107.0|[, black, lives, ...|[, black, lives, ...|(55598,[1,348,933...|(55598,[1,348,933...|[33.1792779099131...|[0.99999999997896...| 0.0|
 everyday American| to be thankful f...| Positive| 0.0|[, to, be, thankf...|[, thankful, food...|(55598,[1,3,68,24...|(55598,[1,3,68,24...|[29.3254033601990...|[0.99657165350978...| 0.0|
 gold peaked the...|now once again on...| Negative| 1.0|[now, once, again...|[downward, trajec...|(55598,[6213,2046...|(55598,[6213,2046...|[5.96858931338679...|[1.76899878355706...| 2.0|
 grocery store wo...| truckers| those who have l...|491.0| [, truckers]| [, truckers]|(55598,[1,2040],[...|(55598,[1,2040],[...|[-7.7253658261761...|[1.47134168302198...| 3.0|
 make no plans| stay inside| wash your hands....|516.0| [, stay, inside]| [, stay, inside]|(55598,[1,51,561]...|(55598,[1,51,561]...|[25.1923779948695...|[0.99999991151717...| 0.0|
 neighbor| etc. from gettin...| Positive| 0.0|[, etc., from, ge...|[, etc., getting,...|(55598,[1,109,641...|(55598,[1,109,641...|[12.3629812699901...|[0.52043096036483...| 0.0|
 no heavy cream(t...| no toast or bread| no flour. At lea...|348.0|[, no, toast, or,...| [, toast, bread]|(55598,[1,590],[1...|(55598,[1,590],[1...|[18.6660206536930...|[0.99998549680479...| 0.0|
 or die a slow de...| but recover slow...| Negative| 1.0|[, but, recover, ...|[, recover, slowl...|(55598,[1,2654],[...|(55598,[1,2654],[...|[-4.2489282493100...|[4.99927163876216...| 2.0|
 orderlies| delivery drivers| teachers and gro...|445.0|[, delivery, driv...|[, delivery, driv...|(55598,[1,61,262]...|(55598,[1,61,262]...|[18.6262944684196...|[0.96190896274923...| 0.0|
 pasta| any kind of fres...| potatoes &amp; v...|393.0|[, any, kind, of,...|[, kind, fresh, m...|(55598,[1,411,553...|(55598,[1,411,553...|[3.54669510923664...|[2.79418158076796...| 3.0|
 stock shelves| pump oil| staff our hospitals|431.0| [, pump, oil]| [, pump, oil]|(55598,[1,46,1849...|(55598,[1,46,1849...|[-9.9943312671329...|[7.69846568228308...| 2.0|
 supermarket workers| lorry drivers an...| Positive| 0.0|[, lorry, drivers...|[, lorry, drivers...|(55598,[1,7,17,21...|(55598,[1,7,17,21...|[27.3584178305617...|[0.00134701812279...| 1.0|
 we return to the...| pharmacy| or grocery store|370.0| [, pharmacy]| [, pharmacy]|(55598,[1,535],[1...|(55598,[1,535],[1...|[19.9799786042994...|[0.99983324108815...| 0.0|
 you will need to...| you can order me...| Extremely Positive| 3.0|[, you, can, orde...|[, order, medicat...|(55598,[1,150],[1...|(55598,[1,150],[1...|[9.02465615671583...|[0.31518878277597...| 2.0|
 16-03-2020| """Like me| Im sure you hav...| 80.0| ["""like, me]| ["""like]| (55598,[],[])| (55598,[],[])|[7.42820383007167...|[0.01221789256166...| 2.0|
 16-03-2020|"Cashier at groce...| Positive| 0.0|["cashier, at, gr...|["cashier, grocer...|(55598,[4,6,18,59...|(55598,[4,6,18,59...|[18.2649789676385...|[3.64016329503881...| 2.0|


### Model Evaluation

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
accuracy

Out[53]: 0.3627757512593365